In [6]:
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import math
from numpy import linalg as LA
cvxopt_solvers. options['show_progress'] = False

class SVRCustom:
    def __init__(self, C=5, gamma = 0.01, eps=0.01):
      self.x_train = None
      self.y_train = None
      self.C = C
      self.gamma = gamma
      self.eps = eps
      self.b = 0
      self.alpha = None
      self.alpha_star = None

    def kernel_rbf(self, x, y, gamma):
        k = math.exp((-1) * gamma * (LA.norm(x-y) * LA.norm(x-y)))
        return k

    # this function should construct a matrix with size (n,n) where each entry is th similarity between the 2 vectors (Xi, Xj)
    def kernel_matrix_rbf(self, X, gamma):
        n = len(X)
        k = np.zeros([n,n])
        for i in range(n):
            for j in range(n):
              k[i][j] = kernel_rbf(X[i], X[j], gamma)
        return k

    def P_Matrix_rbf(self, X, gamma):
        k = kernel_matrix_rbf(X, gamma)
        neg_k = (-1) * k
        temp1 = np.concatenate((k, neg_k))
        temp2 = np.concatenate((neg_k, k))
        P = np.concatenate((temp1, temp2), axis = 1)
        return P

    def q_matrix(self, y, eps):
        temp1 = -y + eps
        temp2 = y + eps
        q = np.concatenate((temp1, temp2))
        return q

    def G_matrix(self, n):
        temp1 = np.identity(2*n)
        temp2 = (-1.0) * temp1
        G = np.concatenate((temp1, temp2))
        return G

    def h_matrix(self, C, n):
        temp1 = C * np.ones(2*n)
        temp2 = np.zeros(2*n)
        h = np.concatenate((temp1, temp2))
        return h

    def A_matrix(self, n):
        temp1 = np.ones(n)
        temp2 = (-1.0) * temp1
        A = np.concatenate((temp1, temp2))
        A = A.reshape(1, -1)
        return A

    def b_matrix(self):
        b = np.zeros(1)
        return b

    # defining a function that calculates the inner summation term (inner loop) of the intercept term
    def b_term_rbf(x_support_vector, n, alpha, alpha_star, x_train, gamma):
        total = 0
        for i in range(n):
            total = total + (alpha[i][0] - alpha_star[i][0]) * kernel_rbf(x_support_vector, x_train[i], gamma)
        return total

    def fit(self, X_train, y_train):
        self.x_train = X_train
        self.y_train = y_train

        n = len(X_train)
        P = P_Matrix_rbf(X_train.values, self.gamma)
        q = q_matrix(y_train, self.eps)
        G = G_matrix(n)
        h = h_matrix(self.C, n)
        A = A_matrix(n)
        b = b_matrix()

        P = cvxopt_matrix(P)
        q = cvxopt_matrix(q)
        G = cvxopt_matrix(G)
        h = cvxopt_matrix(h)
        A = cvxopt_matrix(A)
        b = cvxopt_matrix(b)

        sol = cvxopt_solvers.qp(P, q, G, h, A, b)
        alphas = np.array(sol['x'])

        l= int(len(alphas)/2)
        self.alpha = alphas[0:l, :]
        self.alpha_star = alphas[l: , :]
        
        # get the indices of the support vectors
        supp_vector = []
        for j in range(len(self.alpha)):
            current_alpha_value = self.alpha[j][0]
            if (current_alpha_value > 1e-5 and current_alpha_value < c) :
                supp_vector.append(j)

        # calculation of b
        Y = y_train.values.flatten()
        for i in supp_vector:
            self.b = self.b + Y[i] - eps - b_term_rbf(X_train.values[i], n, self.alpha, self.alpha_star, X_train.values, self.gamma) 
        self.b = self.b/(len(supp_vector))


        def predicted(X_to_be_predicted):
            n = len(self.x_train)
            total = 0
            for i in range(n):
                total = total + (self.alpha[i][0] - self.alpha_star[i][0])*kernel_rbf(X_to_be_predicted, self.x_train[i], self.gamma)
                y = total + self.b

            predictions_SVR = []
            for i in range(len(self.x_train)):
                current_point_pred = Y_predicted_rbf(self.x_train.values[i], n, self.alpha, self.alpha_star, self.x_train.values, b, self.gamma)
                predictions_SVR.append(current_point_pred)
            predictions_SVR = np.array(predictions_SVR)
            return predictions_SVR

import pandas as pd
from sklearn.svm import SVR

train_data = pd.read_csv('train_data.csv')

X = train_data.drop(columns=['SalePrice'])
y = train_data[['SalePrice']]
c = 100
gamma = 0.1
eps = 0.1

modelCustom = SVRCustom(c, eps, gamma)
modelCustom.fit(X.values,y.values.flatten())
predCustom = modelCustom.predicted(X.values)
print(predCustom)

svr_sklearn = SVR(kernel = 'rbf', C = c, gamma = gamma, epsilon= eps)

fitted_SVR_Sklearn = svr_sklearn.fit(X.values, y.values.flatten())
pred = fitted_SVR_Sklearn.predict(X.values)
print(pred)

NameError: name 'P_Matrix_rbf' is not defined

In [3]:
%ls

 Volume in drive D is New Volume
 Volume Serial Number is D09F-D8F3

 Directory of D:\GitHub\ML_From_Scratch\ML_From_Scratch\SVM

08/06/2024  04:13 PM    <DIR>          .
08/06/2024  02:41 PM    <DIR>          ..
08/06/2024  04:13 PM    <DIR>          .ipynb_checkpoints
08/06/2024  04:12 PM             6,365 fd.ipynb
08/06/2024  03:50 PM             4,266 SVR.py
08/06/2024  03:50 PM               554 train.py
08/05/2024  10:44 PM           654,662 train_data.csv
               4 File(s)        665,847 bytes
               3 Dir(s)  459,845,083,136 bytes free
